In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
import os
from nltk.tokenize import sent_tokenize
import pandas as pd
import numpy as np
stemmer = PorterStemmer() #or SnowballStemmer(language='english')
import pickle

In [ ]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
files = os.listdir("../data/LEGAL_TEXT/")
print(len(files))
master = pd.DataFrame(columns=['text','index','tid'])

In [ ]:
for i in range(len(files)):
    print(" --------- #",i+1,": FILE ",files[i]," --------")
    TID = (files[i])[:-4]
    print(TID)
    data = open("../data/LEGAL_TEXT/"+files[i],'r')
    d = data.read()
    d = re.sub(r'\n+','.',d)
    d = re.sub(r' +',' ',d)
    data = sent_tokenize(d)
    df = pd.DataFrame(columns=['text','index','tid'])
    for j in range(len(data)):
        data[j] = re.sub(r'\.+',' ',data[j])
        data[j] = re.sub(r',+','',data[j])
        if data[j]=='' or data[j]==' ':
            data[j]='none'
        df.loc[j,'text'] = data[j]
        df.loc[j,'index'] = j
        df.loc[j,'tid'] = TID
    df = df[df.text!='none'].reset_index(drop=True)
    for j in range(len(df)):
        df.loc[j,'index']=str(j)
    master = master.append(df)
    #print(df.head(10))

In [ ]:
print(len(master))
for i in range(len(master)):
    master.loc[i,'index'] = i
master.to_csv("FOR_TF_IDF.csv",index=False)

In [ ]:
master.head(1000)

In [ ]:
master = master.reset_index(drop=True)

In [ ]:
df=pd.read_csv("FOR_TF_IDF.csv")
df=df.reset_index(drop=True)
print(len(df))

In [ ]:
doc_sample = str(df.loc[100000,'text'])
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

In [ ]:
processed_docs = df['text'].map(preprocess)

In [ ]:
len(processed_docs)

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
dictionary.filter_extremes(no_below=3, no_above=0.65, keep_n=1000000)

In [ ]:
print(dictionary)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[1]

In [ ]:
bow_doc_1 = bow_corpus[1]
for i in range(len(bow_doc_1)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_1[i][0], 
                                               dictionary[bow_doc_1[i][0]], 
bow_doc_1[i][1]))

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
counter=0
for doc in corpus_tfidf:
    print("===================== NUMBER: ",counter+1)
    counter+=1
    pprint(doc)
    break

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
for index, score in sorted(lda_model[bow_corpus[1]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

In [ ]:
for index, score in sorted(lda_model_tfidf[bow_corpus[1]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

In [ ]:
master = pd.read_csv("FOR_TF_IDF.csv")
master=master.reset_index(drop=True)
import re
for i in range(len(master)):
    print("------ LINE ",i+1,' OF ',len(master)," ------")
    unseen_document = master.loc[i,'text']
    bow_vector = dictionary.doc2bow(preprocess(unseen_document))
    x=""
    for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
        print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))
        x = lda_model.print_topic(index, 10)
        x = re.sub(r'\*',' ',x)
        x = re.sub(r'[0-9]','',x)
        x = re.sub(r'\+',' ',x)
        x = re.sub(r'\.',' ',x)
        x = re.sub(r'"',' ',x)
        x = re.sub(r' +',' ',x)
        x = re.sub(r' ',',',x)
        break
    master.loc[i,'tokens'] = x
    print(x)